## Caso COVID-19 Fase 1
Julián Correa, Guillermo Cortés, Cristian Sarmiento

## Creditos de la información
La data utilizada en el ejercicio es tomada del **repositorio de datos COVID-19 del Centro de Ciencias de Datos e Ingeniria de Sistemas (CSSE) de la Universidad Johns Hopkins (https://github.com/CSSEGISandData/COVID-19)[https://github.com/CSSEGISandData/COVID-19]** 

### Preprocesarmiento de información

In [1]:
## import libraries and dependencies
import pandas as pd

## read downloaded csv file
covid_cases = pd.read_csv('time_series_covid19_confirmed_global.csv')
covid_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


Al realizar la inspección nos encontramos con que la tabla esta en formato horizontan donde las fechas son columnas en el data frame lo que hace complicado el procesamiento de la información. Procedemos hacer el comando **melt** de pandas para convertir **date** en variable y **value** como otra variable que contiene el numero de casos.

In [2]:
## melt dataframe to get dates
id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long']
df = covid_cases.melt(id_vars=id_vars, var_name='date')
df.sample(n=5)

,Province/State,Country/Region,Lat,Long,date,value
40561,NaN,Cyprus,35.126400,33.42990,6/10/20,972
78796,NaN,Monaco,43.733300,7.41670,10/20/20,271
104831,NaN,Papua New Guinea,-6.314993,143.95555,1/18/21,834
204890,NaN,United Kingdom,55.378100,-3.43600,12/30/21,12748050
15560,NaN,Sudan,12.862800,30.21760,3/15/20,1


## Revisión de tipos de variables

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330327 entries, 0 to 330326
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Province/State  104013 non-null  object 
 1   Country/Region  330327 non-null  object 
 2   Lat             328041 non-null  float64
 3   Long            328041 non-null  float64
 4   date            330327 non-null  object 
 5   value           330327 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 15.1+ MB


Encontramos que la variable **date** no corresponde al tipo de valor de fecha, procedemos hacer el ajuste convirtiendo **object** a tipo **datetime64**

In [4]:
## Transform date column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
df.sample(n=5)

,Province/State,Country/Region,Lat,Long,date,value
22370,NaN,Ethiopia,9.1450,40.4897,2020-04-08,55
150320,Alberta,Canada,53.9333,-116.5765,2021-06-25,231641
23141,NaN,Bangladesh,23.6850,90.3563,2020-04-11,482
320432,NaN,Romania,45.9432,24.9668,2023-02-03,3325006
161661,NaN,Ecuador,-1.8312,-78.1834,2021-08-03,489089


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330327 entries, 0 to 330326
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Province/State  104013 non-null  object        
 1   Country/Region  330327 non-null  object        
 2   Lat             328041 non-null  float64       
 3   Long            328041 non-null  float64       
 4   date            330327 non-null  datetime64[ns]
 5   value           330327 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 15.1+ MB


## 1. ¿En cuál mes se presentó el mayor número de contagios?

In [6]:
#df['month'] = df['date'].apply(lambda x: x.month)
df['month'] = df['date'].dt.month 
df['year'] = df['date'].dt.year
df_agg_month = df.groupby(['month', 'year'], as_index=False).agg({'value': 'sum'})
df_agg_month

,month,year,value
0,1,2020,38557
1,1,2021,2924193988
2,1,2022,10275647283
3,1,2023,20669212563
4,2,2020,1672160
5,2,2021,3062477618
6,2,2022,11575136242
7,2,2023,18853048685
8,3,2020,8977762
9,3,2021,3768712195


In [7]:
df_agg_month[df_agg_month['value'] == df_agg_month['value'].max()]

,month,year,value
3,1,2023,20669212563


#### 2. ¿En ese mismo mes, cuál fue el país que reportó más contagios?

In [8]:
max_month = df_agg_month.iloc[0]['month']
max_year = df_agg_month.iloc[0]['year']
cond = ((df['month'] == max_month) & (df['year'] == max_year))
df_country = df[cond].groupby('Country/Region', as_index=False).agg({'value':'sum'})

df_country[df_country['value'] == df_country['value'].max()]

,Country/Region,value
37,China,38008


#### 3. ¿Cuál es el país con el menor número de casos reportados hasta la fecha?

In [9]:
df_min_country = df.groupby('Country/Region', as_index=False).agg({'value':'sum'})
df_min_country[(df_min_country['value'] == df_min_country['value'].min())]

,Country/Region,value
93,"Korea, North",300


***
___